<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/deepseek-llm-7b-base", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-05 17:08:12.785616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749143292.996019      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749143293.054855      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

deepseek-ai/deepseek-llm-7b-base does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.9 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


<a name="Data"></a>
### Data Prep
**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [2]:
import json
from datasets import Dataset
# Load PubMedQA dataset
def load_pubmedqa_data(file_path):
    """Load and parse PubMedQA dataset from JSON file"""
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    instructions = []
    inputs = []
    outputs = []
    
    for paper_id, paper_data in data.items():
        question = paper_data.get("QUESTION", "")
        contexts = paper_data.get("CONTEXTS", [])
        long_answer = paper_data.get("LONG_ANSWER", "")
        final_decision = paper_data.get("final_decision", "")
        reasoning_required = paper_data.get("reasoning_required_pred", "no")
        
        # Create context string from all contexts
        context_text = " ".join(contexts) if contexts else ""
        
        # Create instruction based on whether reasoning is required
        if reasoning_required == "yes":
            instruction = "Answer the following medical question with detailed reasoning. Provide your reasoning process and then give a clear yes/no answer."
        else:
            instruction = "Answer the following medical question clearly and concisely with a yes/no answer and brief explanation."
        
        # Create input with question and context
        input_text = f"Question: {question}\n\nContext: {context_text}"
        
        # Create output with reasoning and answer
        if reasoning_required == "yes":
            output_text = f"Reasoning: {long_answer}\n\nAnswer: {final_decision}"
        else:
            output_text = f"Answer: {final_decision}\n\nExplanation: {long_answer}"
        
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(output_text)
    
    return {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

# Load your PubMedQA dataset
pubmedqa_data = load_pubmedqa_data("/kaggle/input/pubmedqa/ori_pqal.json")
dataset = Dataset.from_dict(pubmedqa_data)
# Get the last 100 entries as test data
test_data = dataset.select(range(len(dataset) - 100, len(dataset)))

# Save to JSON
test_data_dict = test_data.to_dict()
with open("pubmedqa_test_last100.json", "w") as f:
    json.dump(test_data_dict, f, indent=2)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

## WandB for logging

In [ ]:
!pip install wandb -q

In [7]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
WANDB_API_KEY = user_secrets.get_secret("WANDB_API")

# Login to wandb using the API key
wandb.login(key=WANDB_API_KEY)

# Initialize wandb project
wandb.init(
    project="pubmedqa-finetuning",
    name="unsloth/deepseek-7b-bnb-4bit",
    config={
        "model": "unsloth/deepseek-7b-bnb-4bit",
        "dataset": "pubmedqa",
        "max_seq_length": 2048,
        "lora_r": 16,
        "lora_alpha": 16,
        "learning_rate": 2e-4,
        "max_steps": 60,
        "batch_size": 1,
        "gradient_accumulation_steps": 8,
    }
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rauniyaravishek72 (rauniyaravishek72-amrita-vishwa-vidyapeetham) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
        run_name = "unsloth/deepseek-7b-bnb-4bit"),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainerCallback

class WandBCallback(TrainerCallback):
    def on_log(self, args, state, control, model=None, logs=None, **kwargs):
        if logs:
            wandb.log(logs)

# Add callback to trainer
trainer.add_callback(WandBCallback())

In [10]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla P100-PCIE-16GB. Max memory = 15.888 GB.
6.6 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 3 | Total steps = 186
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 37,478,400/7,000,000,000 (0.54% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.980200
2,1.953100
3,2.014600
4,1.935500
5,2.004800
6,1.939900
7,1.814800
8,1.755100
9,1.740000
10,1.597000


In [12]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

9795.9051 seconds used for training.
163.27 minutes used for training.
Peak reserved memory = 6.863 GB.
Peak reserved memory for training = 0.263 GB.
Peak reserved memory % of max memory = 43.196 %.
Peak reserved memory for training % of max memory = 1.655 %.


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [13]:
from peft import PeftModel
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/deepseek-llm-7b-base",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# ✅ Tell PEFT this is a local folder
model = PeftModel.from_pretrained(model, "/kaggle/working/outputs/checkpoint-186", is_trainable=False)

# ✅ Now you can safely merge
model = model.merge_and_unload()


==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

deepseek-ai/deepseek-llm-7b-base does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [14]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
model.save_pretrained("deepseek_lora_model")
tokenizer.save_pretrained("deepseek_lora_model")
model.push_to_hub("Avishek8136/deepseek_lora_model", token=user_secrets.get_secret("HF"))
tokenizer.push_to_hub("Avishek8136/deepseek_lora_model", token=user_secrets.get_secret("HF"))

README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/4.81G [00:00<?, ?B/s]

Saved model to https://huggingface.co/Avishek8136/deepseek_lora_model


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
from unsloth import FastLanguageModel
import torch
dtype = torch.float16

# Model loading parameters
model_name = "Avishek8136/deepseek_lora_model"  # Replace with your actual model path
max_seq_length = 2048      # Set your actual max sequence length
dtype = None             # Can be "auto", "float16", etc.
load_in_4bit = True        # Set to True or False based on your setup

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Enable faster inference
FastLanguageModel.for_inference(model)

# Define the Alpaca prompt format
alpaca_prompt = (
    "### Instruction:\n{0}\n\n"
    "### Input:\n{1}\n\n"
    "### Response:\n{2}"
)

# Format the prompt
prompt = (
    "Question: Do statins reduce the risk of stroke?\n"
    "Context: Statins have been shown in several studies to reduce the risk...\n"
    "Answer:"
)


# Tokenize the prompt
inputs = tokenizer(
    [prompt],
    return_tensors="pt"
).to("cuda")

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    use_cache=True
)

# Decode and print result
decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_output[0])

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-05 20:22:52.907294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749154973.093737      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749154973.147914      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.50M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/355 [00:00<?, ?B/s]

Question: Do statins reduce the risk of stroke?
Context: Statins have been shown in several studies to reduce the risk...
Answer: Yes, statins reduce the risk of stroke.
The evidence for this is based on a meta-analysis of 11 studies, which included 10,000 patients. The results showed that statins reduced the risk of stroke by 25%.
The mechanism by which statins reduce the


## Evaluation

In [5]:
!pip install -q evaluate bert-score rouge-score nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00


In [6]:
import evaluate

# Load metrics
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
bleu = evaluate.load("bleu")

# Collect predictions and references
predictions = []
references = []

In [7]:
for item in test_data:
    question = item['input'].split('\n')[0].replace("Question: ", "")
    context = item['input'].split('\n')[2].replace("Context: ", "")
    
    # Build the prompt exactly as you want
    prompt = f"Question: {question}\nContext: {context}\nAnswer:"
    
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    # Extract generated answer after the prompt text
    generated_answer = decoded[len(prompt):].strip()
    
    predictions.append(generated_answer)
    references.append(item['output'])

In [8]:
# Compute metrics
rouge_results = rouge.compute(predictions=predictions, references=references)
bertscore_results = bertscore.compute(predictions=predictions, references=references, lang="en")
bleu_results = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

# Display results
print("ROUGE:", rouge_results)
print("BERTScore:", bertscore_results)
print("BLEU:", bleu_results)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE: {'rouge1': 0.15473193802018925, 'rouge2': 0.052361977566690476, 'rougeL': 0.11420220394678712, 'rougeLsum': 0.11974978453622162}
BERTScore: {'precision': [0.7766854166984558, 0.7945622801780701, 0.8484569191932678, 0.7979864478111267, 0.8043580055236816, 0.8411756753921509, 0.787905216217041, 0.7964439392089844, 0.8524954319000244, 0.8766778707504272, 0.8218292593955994, 0.7939736843109131, 0.8570103645324707, 0.7957786917686462, 0.8543639183044434, 0.8402775526046753, 0.8072023391723633, 0.8090783357620239, 0.8073247671127319, 0.8469087481498718, 0.7743427753448486, 0.8188636302947998, 0.7469717264175415, 0.8322058320045471, 0.7874513864517212, 0.8790065050125122, 0.8093195557594299, 0.9276460409164429, 0.7544799447059631, 0.7957675457000732, 0.8249865770339966, 0.8389310836791992, 0.8246883749961853, 0.8222693204879761, 0.8277260065078735, 0.8096894025802612, 0.82369065284729, 0.8023426532745361, 0.7632513046264648, 0.8306597471237183, 0.779066264629364, 0.8370758295059204, 0.

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [9]:
# Format the prompt
prompt = (
    "Question: Do statins reduce the risk of stroke?\n"
    "Context:"
    "Answer:"
)


# Tokenize the prompt
inputs = tokenizer(
    [prompt],
    return_tensors="pt"
).to("cuda")

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    use_cache=True
)

# Decode and print result
decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_output[0])

Question: Do statins reduce the risk of stroke?
Context:Answer: Yes, statins reduce the risk of stroke.
Statins are a class of drugs that lower cholesterol levels in the blood. They are commonly used to prevent heart disease and stroke.
Statins have been shown to reduce the risk of stroke in several studies. One study, for example, found that people who took


In [10]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [11]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [12]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>